In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
import json
from google.colab import files
drive.mount('/content/drive')

pd.set_option('max_colwidth', 99999)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/ML Caption Craft/Influencer brand dataset/post_info.txt', sep='\t', header=None, names=['id','user' ,'is_sponsored','json_p' , 'img_p'])
data.head()

In [ ]:
#filter by only one image in img_p
print(data.dtypes)
# split the string in img_p, and count the number of images
data['image_count'] = data['img_p'].apply(
    lambda row: len(str(row)[1:-1].split(','))
)
data = data[data["image_count"] == 1]
# restructure the string by removing additional square brackets and single quotes
data['img_p'] = data['img_p'].apply(
    lambda row: str(row)[2:-2]
)
data.head()

In [ ]:
# confirm that there are no multi-image posts
data['image_count'].plot()

In [ ]:
#filtered posts by non_sponsored, as we are only interested in non-sponsored posts
non_sponsored_posts_df = data[data["is_sponsored"] == 0]
non_sponsored_posts_df.head()

In [ ]:
# unzip json files to be read for extraction
!unzip '/content/drive/MyDrive/ML Caption Craft/Influencer brand dataset/json_files.zip'
!ls -la

Streaming output truncated to the last 5000 lines.
  inflating: json/1973349931071017124.json  
  inflating: json/1804623845716325143.json  
  inflating: json/1750458824093315141.json  
  inflating: json/1615455682536952879.json  
  inflating: json/1849329892418539381.json  
  inflating: json/1741711284929641181.json  
  inflating: json/1404532811544933797.json  
  inflating: json/1896159802804297957.json  
  inflating: json/1674166334681035320.json  
  inflating: json/1811567332326911800.json  
  inflating: json/1786637894493922676.json  
  inflating: json/1897547142487549489.json  
  inflating: json/1968093970673543373.json  
  inflating: json/1855458587008888774.json  
  inflating: json/1797290405870429629.json  
  inflating: json/2025063461469752707.json  
  inflating: json/1930105549284006879.json  
  inflating: json/1768052083955027803.json  
  inflating: json/1977138122634433202.json  
  inflating: json/1999758234475279410.json  
  inflating: json/1672160071544514376.json  
  in

In [ ]:
print(non_sponsored_posts_df.shape)

In [ ]:
# open json files and extract caption, like_count, comment_count, is_video, add to dataframe
def get_json_content(path):
 
  filepath = f"/content/json/{path}"
  try:
    f = open(filepath, 'r')
    data = json.load(f)

    caption_location = data["edge_media_to_caption"]["edges"]
    if len(caption_location) != 0:
      caption = caption_location[0]["node"]["text"]
    else:
      caption = ""

    

    like_count = data["edge_media_preview_like"]["count"] if "count" in data["edge_media_preview_like"] and "edge_media_preview_like" in data else ""

    comment_count = data["edge_media_to_comment"]["count"] if "count" in data["edge_media_to_comment"] and "edge_media_to_comment" in data else ""
    is_video = data["is_video"] if "is_video" in data else ""
    f.close()
    return caption, like_count, comment_count, is_video

  except(json.JSONDecodeError, FileNotFoundError) as e:
    return np.nan, np.nan, np.nan, np.nan
  except(IndexError):
    print("index error at ", filepath)
    return np.nan, np.nan, np.nan, np.nan

In [ ]:
#sample json
get_json_content('1857934737658773774.json')

('', 153, 6, False)

In [ ]:
from tqdm import tqdm
tqdm.pandas()


In [ ]:
for index, row, in tqdm(non_sponsored_posts_df.iterrows()):

  caption, like_count, comment_count, is_video = get_json_content(row['json_p'])
  non_sponsored_posts_df.loc[index, 'caption'] = caption
  non_sponsored_posts_df.loc[index,'like_count'] = like_count
  non_sponsored_posts_df.loc[index,'comment_count'] = comment_count
  non_sponsored_posts_df.loc[index,'is_video'] = is_video


non_sponsored_posts_df.to_csv("fully_processed_dataset.csv")
files.download("fully_processed_dataset.csv")

595933it [2:33:32, 64.69it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>